In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, date, time, timedelta
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# def to_time(str_time):
#     if len(str_time) >= 7:
#         date_time_str = str_time
#         date_time_obj = datetime.datetime.strptime(date_time_str, '%H:%M:%S')
#         return date_time_obj.time()
#     else:
#         print(str_time)
#         date_time_str = '0:00:00'
#         date_time_obj = datetime.datetime.strptime(date_time_str, '%H:%M:%S')
#         return date_time_obj.time()
#
#
# def hour_minutes(duration):
#     duration = to_time(duration)
#     days, seconds = duration.days, duration.seconds
#     hours = days * 24 + seconds // 3600
#     minutes = (seconds % 3600) // 60
#     return hours, minutes


In [ ]:
df = pd.read_csv('call_center_year.csv')

print(df.head(10))

Функция преобразования строковых даты и времени в  datetime

In [ ]:
def to_datetime(str_time, str_date):
    if len(str_time) >= 7:
        date_time_str = str_time
    else:
        print(str_time)
        date_time_str = '0:00:00'
    date_time_obj = datetime.strptime(date_time_str, '%H:%M:%S')
    t = date_time_obj.time()    
    date = int(str_date)
    day = date % 100
    date //= 100
    month = date % 100
    year = 1900 + date // 100
    d = datetime(year, month, day)
    return datetime.combine(d, t)

 Преобразовываем все столбцы с временами в datetime 

In [ ]:
df_copy = df.copy()

ser_datetime_q_start = []
ser_datetime_q_exit = []
ser_datetime_vru_entry = []
ser_datetime_vru_exit = []
ser_datetime_ser_start = []
ser_datetime_ser_exit = []

for index, row in df_copy.iterrows():
    ser_datetime_q_start.append(to_datetime(row['q_start'], row['date']))
    ser_datetime_q_exit.append(to_datetime(row['q_exit'], row['date']))
    ser_datetime_vru_entry.append(to_datetime(row['vru_entry'], row['date']))
    ser_datetime_vru_exit.append(to_datetime(row['vru_exit'], row['date']))
    ser_datetime_ser_start.append(to_datetime(row['ser_start'], row['date']))
    ser_datetime_ser_exit.append(to_datetime(row['ser_exit'], row['date']))

    
df_copy['q_start_dt'] = ser_datetime_q_start
df_copy['q_exit_dt'] = ser_datetime_q_exit
df_copy['vru_entry_dt'] = ser_datetime_vru_entry
df_copy['vru_exit_dt'] = ser_datetime_vru_exit
df_copy['ser_start_dt'] = ser_datetime_ser_start
df_copy['q_ser_exitt'] = ser_datetime_ser_exit

df_copy.head(15)

df_copy.to_csv('call_center_year_dt.csv')


#если время в очереди 0, то смотрим вру тайм

Check if there dependence which operator will answer on the type of clients call 

In [ ]:
PS = df_copy.loc[(df_copy['type'] == 'PS')]['server'].unique()
PE = df_copy.loc[(df_copy['type'] == 'PE')]['server'].unique()
IN = df_copy.loc[(df_copy['type'] == 'IN')]['server'].unique()
NE = df_copy.loc[(df_copy['type'] == 'NE')]['server'].unique()
NW = df_copy.loc[(df_copy['type'] == 'NW')]['server'].unique()
TT = df_copy.loc[(df_copy['type'] == 'TT') | (df_copy['type'] == ' TT')]['server'].unique()

In [ ]:
from venn import venn

sets = { 'PS - regular activity':set(PS),
         'PE - regular activity in English ':set(PE), 
         'IN -  internet consulting':set(IN), 
         'NE - stock exchange activity':set(NE),
         'NW - potential customer getting information':set(NW), 
         'TT -  customers who left a message asking the bank to return their call ':set(TT)}
fig, ax = plt.subplots(1, figsize=(20,20))
venn(sets, ax=ax)
plt.show()

It means, that 20 opearators answer on every type of call, 9 operators onswer on every type of call, but don't speak english, 12 operators advise only about internet colsulting.
Other operators answer of multiple type of calls.

In [ ]:
df_no_server = df_copy.loc[df_copy['server'] == 'NO_SERVER']

In [ ]:
len(df_copy)

In [ ]:
len(df_no_server)

In [ ]:
print(df_no_server)

In [ ]:
print(len(df_no_server.loc[(df_no_server['vru_time'] == 0) & (df_no_server['q_time'] != 0)]))

In [ ]:
print(df_no_server.loc[(df_no_server['vru_time'] == 0)])

Выбросы

In [ ]:
df_copy.boxplot(column=['q_time'])

In [ ]:
df_copy['q_time'].describe()

Уберем значения в q_time, которые больше 5000

In [ ]:
df_copy = df_copy.loc[df_copy['q_time'] < 5000]
df_copy.boxplot(column=['q_time'])

In [ ]:
df_copy['q_time'].describe()

In [ ]:
df_copy_2 = df_copy.loc[df_copy['q_time'] < 2000]
df_copy_2.boxplot(column=['q_time'])

In [ ]:
df_copy_2['q_time'].describe()

In [ ]:
values = df_copy_2['q_time'].value_counts()

Делим по промежуткам в 30 минут и вычисляем количество человек в очереди и работников колцентра

In [ ]:
def to_date(str_date):
    date = int(str_date)
    day = date % 100
    date //= 100
    month = date % 100
    year = 1900 + date // 100
    return [year, month, day]

In [ ]:
type(df_copy['vru_entry_dt'][0])

In [ ]:
df_copy = df_copy.loc[df_copy['q_time'] < 5000]

In [ ]:
df_copy_2['q_time'].describe()

In [ ]:
values = df_copy_2['q_time'].value_counts()

Делим по промежуткам в 30 минут и вычисляем количество человек в очереди и работников колцентра

In [ ]:
def to_date(str_date):
    date = int(str_date)
    day = date % 100
    date //= 100
    month = date % 100
    year = 1900 + date // 100
    return [year, month, day]

In [ ]:
type(df_copy['vru_entry_dt'][0])

In [ ]:
df_copy_2 = df_copy.loc[(df_copy['q_time'] < 2000) & (df_copy['vru_time'] < 2000)]

In [ ]:
def diff_time_interval(n):
    """
    input:
    n - interval in minutes
    """

    all_datas = df_copy_2['date'].unique()
    means_x = []
    means_y = []
    means_op_x = []
    means_op_y = []
    medians_x = []
    medians_y = []
    medians_op_x = []
    medians_op_y = []
    stds_x = []
    stds_y = []
    stds_op_x = []
    stds_op_y = []
    hu_means_x = [] #hang up
    hu_means_y = []
    hu_num_x = []
    hu_num_y = []
    
    
    df_copy_3 = df_copy_2

    if n == 15:
        treshold_op = 10
        treshold_c = 85
    if n == 30:
        treshold_op = 14
        treshold_c = 120
    if n == 60:
        treshold_op = 25
        treshold_c = 250
        
        
    for day in all_datas:
        df_day = df_copy_3.loc[df_copy_3['date'] == day]
        date_type = to_date(day)
        base = datetime(date_type[0], date_type[1], date_type[2])
        arr_times = [base + timedelta(minutes=i) for i in range(420, 1440, n)]
        for i in range(len(arr_times)-1):
            df_n_min = df_day.loc[(df_day['vru_entry_dt'] > arr_times[i]) & (df_day['vru_entry_dt'] < arr_times[i+1])]
            df_n_min_no_server = df_n_min.loc[df_n_min['server'] == 'NO_SERVER']
            n_clients = df_n_min.shape[0]
            n_operators = df_n_min['server'].nunique()
            n_off_clients = df_n_min_no_server.shape[0]
            if n_operators == 0:
                n_operators = 1
            
            if df_n_min['q_time'].mean() != 0:
                if n_clients <= treshold_c:
                    means_y.append(df_n_min['q_time'].mean())
                    means_x.append(n_clients)
                if n_clients/n_operators <= treshold_op:
                    means_op_y.append(df_n_min['q_time'].mean())
                    means_op_x.append(n_clients/n_operators)
                    if df_n_min_no_server['q_time'].mean() < 500 and df_n_min_no_server['q_time'].mean() != 0:
                        hu_means_y.append(df_n_min_no_server['q_time'].mean())
                        hu_means_x.append(n_clients/n_operators)
                    hu_num_y.append(df_n_min_no_server.shape[0])
                    hu_num_x.append(n_clients/n_operators)
            if df_n_min['q_time'].median() != 0:
                if n_clients <= treshold_c:
                    medians_y.append(df_n_min['q_time'].median())
                    medians_x.append(n_clients)
                if (n_clients/n_operators <= treshold_op):
                    medians_op_y.append(df_n_min['q_time'].median())
                    medians_op_x.append(n_clients/n_operators)
            if df_n_min['q_time'].std() != 0:
                if n_clients <= treshold_c:
                    stds_y.append(df_n_min['q_time'].std())
                    stds_x.append(n_clients)
                if (n_clients/n_operators <= treshold_op):
                    stds_op_y.append(df_n_min['q_time'].std())
                    stds_op_x.append(n_clients/n_operators)


    dict_return = {'min': n,
                   'mean_dep_people_x': means_x,
                   'mean_dep_people_y': means_y,
                   'mean_dep_operator_x': means_op_x,
                   'mean_dep_operator_y': means_op_y,
                   'median_dep_people_x': medians_x, 
                   'median_dep_people_y': medians_y,
                   'median_dep_operator_x': medians_op_x,
                   'median_dep_operator_y': medians_op_y,
                   'std_dep_people_x': stds_x, 
                   'std_dep_people_y': stds_y,
                   'std_dep_operator_x': stds_op_x, 
                   'std_dep_operator_y': stds_op_y, 
                   'hu_mean_x': hu_means_x, 
                   'hu_mean_y': hu_means_y, 
                   'hu_num_x': hu_num_x, 
                   'hu_num_y': hu_num_y}

    return dict_return

Функция линейной регрессии для 25%, 50%, 75%

In [ ]:
def lines_regression(X, y):
    tuples = list(zip(X, y))
    tuples = sorted(tuples, key= lambda x : x[0])
    tuples = [tup for tup in tuples if not np.isnan(tup[1])]
    
    #average y
    dict_avg = {}
    X_unique = list(set([tup[0] for tup in tuples]))
    
    for x in X_unique:
        ys = [tup[1] for tup in tuples if tup[0]==x]
        ys_avg = sum(ys) / len(ys)
        dict_avg.update({x : ys_avg})
    
    #linear regression 25% 50% 75%
    X, y = list(map(list, zip(*tuples)))
    percents = [X[0], X[0]+(X[-1]-X[0])*0.25, X[0]+(X[-1]-X[0])*0.5, X[0]+(X[-1]-X[0])*0.75, X[-1]]
    slopes = []
    for i in range(len(percents)-1):
        tuples_now = [tup for tup in tuples if percents[i] < tup[0] and tup[0]<percents[i+1]]
        X_now, y_now = list(map(list, zip(*tuples_now)))
        X_now = np.array(X_now)
        X_now = X_now.reshape(-1,1)
        model = LinearRegression()
        model.fit(X_now, y_now)
        slopes.append(model.coef_) # (slope, shift)
    slopes.append(1)
    return slopes, percents, dict_avg

Линии 25% 50% 75% 

In [ ]:
def lines(x, y, part):
    
#     if t == 30:
        
    
    tuples = list(zip(x, y))
    tuples = sorted(tuples, key= lambda x : x[0])
    tuples = [tup for tup in tuples if not np.isnan(tup[1])]
    
    #average y
    dict_25 = {}
    dict_50 = {}
    dict_75 = {}
    x_unique = sorted(list(set([tup[0] for tup in tuples])))
    step = (x_unique[-1]-x_unique[0])/part
    
    len_y = []
#     for i in np.arange(x_unique[0], x_unique[-1]):
    for i in x_unique:
        y_s = sorted([tup[1] for tup in tuples if i<=tup[0] and tup[0]<=i+step])
        len_y.append(len(y_s))
        all_y_s = len(y_s)-1
        if all_y_s >= 0:

            y_s_50 = y_s[round(all_y_s*0.5)]
            y_s_25 = y_s[round(all_y_s*0.25)]
            y_s_75 = y_s[round(all_y_s*0.75)]


            dict_25.update({i:y_s_25})
            dict_50.update({i:y_s_50})
            dict_75.update({i:y_s_75})
    
    print('average number of dots in intervals:', sum(len_y) / len(len_y) )

    return dict_25, dict_50, dict_75

Функция графика

In [ ]:
def make_plot(x, y, title, t, xlab, ylab):
#     slopes, percents, dict_avg = lines_regression(x, y)
    
    if t==15:
        col = [0, 0.5, 0, 0.05]
        p = 120
    if t == 30:
        col = [1, 0, 0, 0.05]
        p = 60
    if t == 60:
        col = [0, 0, 1, 0.05]
        p = 30
    fig, ax = plt.subplots()

    ax.scatter(x, y,
               color = col)    #  цвет точек
    ax.set_title(title)

#     ax.scatter(dict_avg.keys(), dict_avg.values(),
#                color = [128/225, 0, 64/225, 0.4])
    fig.set_figwidth(16)     #  ширина и
    fig.set_figheight(16)    #  высота "Figure"
    
    dict_25, dict_50, dict_75 = lines(x, y, p)

    ax.plot(dict_75.keys(), dict_75.values(), color = [23/255, 175/255, 209/255, 1], label="75th percentile")
    ax.plot(dict_50.keys(), dict_50.values(), color = [209/255, 128/255, 23/255, 1], label="50th percentile")
    ax.plot(dict_25.keys(), dict_25.values(), color = [143/255, 62/255, 214/255, 1], label="25th percentile")
    
    ax.legend()
    plt.ylim([-10,max(dict_75.values())*1.1])
    plt.xlabel(xlab)
    plt.ylabel(ylab)
#    plt.xlim(right = max(dict_75.keys())*0.8)
    plt.show()
    
    #linear regression
#     start_line = percents[0]
#     for i in range(len(slopes)-1):
#         slope = slopes[i]
#         xvals = np.linspace(percents[i],percents[i+1],100) #100 points from 0 to 6 in ndarray
#         shift = slope*percents[i] - start_line
#         start_line = slope*percents[i+1]-shift
#         plt.plot(percents[i+1], slope*percents[i+1]-shift, 'o')
#         plt.plot(xvals, slope*xvals-shift, 'purple')

In [ ]:
#d_30_mean_x, d_30_mean_y, d_30_median_x, d_30_median_y, d_30_mean_op_x, d_30_mean_op_y, d_30_median_op_x, d_30_median_op_y, d_30_std_x, d_30_std_y, d_30_std_op_x, d_30_std_op_y, d_30_off_mean_q_x, d_30_off_mean_q_y, d_30_off_num_x, d_30_off_num_y = diff_time_interval(30)

In [ ]:
dict_15 = diff_time_interval(15)
dict_30 = diff_time_interval(30)
dict_60 = diff_time_interval(60)

In [ ]:
for dict_n in [dict_15, dict_30, dict_60]:
    n = dict_n['min']
    make_plot(dict_n['mean_dep_people_x'], dict_n['mean_dep_people_y'], 'mean waiting time depends on people '
              'in queue within %s minutes' % n, n, 'number of people in queue', 'mean waiting time in queue in seconds')
    make_plot(dict_n['mean_dep_operator_x'], dict_n['mean_dep_operator_y'], 'mean waiting time depends on people in queue per operator'
              ' within %s minutes' % n, n, 'number of people in queue per operator', 'mean waiting time in queue in seconds')
    make_plot(dict_n['median_dep_people_x'], dict_n['median_dep_people_y'], 'median waiting time depends on people '
              'in queue within %s minutes' % n, n, 'number of people in queue', 'median waiting time in queue in seconds')
    make_plot(dict_n['median_dep_operator_x'], dict_n['median_dep_operator_y'], 'median waiting time depends on people in queue per operator'
              'within %s minutes' % n, n, 'number of people in queue per operator', 'median waiting time in queue in seconds')
    make_plot(dict_n['std_dep_people_x'], dict_n['std_dep_people_y'], 'standard deviation of waiting time depends on number of people'
              ' in queue within %s minutes' % n, n, 'number of people in queue', 'standard deviation of waiting time in queue in seconds')
    make_plot(dict_n['std_dep_operator_x'], dict_n['std_dep_operator_y'], 'standard deviation of waiting time depends on number of people per operator'
              ' in queue within %s minutes' % n, n, 'number of people in queue per operator', 'standard deviation o waiting time in queue in seconds')
    make_plot(dict_n['hu_mean_x'], dict_n['hu_mean_y'], 'mean waiting time for people before hang up depends on '
              'number of people per operator within %s min' % n, n, 'number of people in queue', 'mean waiting time in queue '
              'before hang up in seconds')
    make_plot(dict_n['hu_num_x'], dict_n['hu_num_y'], 'number of people hang up depends on '
              'number of people per operator within %s min' % n, n,'number of people in queue per operator',
              'number of people hang up')